In [140]:
import spacy
from spacy import displacy
from number_parser import parse_ordinal

In [2]:
%cd /workspace/novel-tongue/notebooks/aligned_texts

/workspace/novel-tongue/notebooks/aligned_texts


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
src_sents = []
tgt_sents = []
with open(f"c1_aligned.txt", "r") as f:
    c1 = f.read()

    lines = c1.splitlines()
    for i in range(0, len(lines), 3):
        src_sents.append(lines[i])
        tgt_sents.append(lines[i+1])

In [52]:
for src_text, tgt_text in zip(src_sents[99:100], tgt_sents[99:100]):
    nlp_src = spacy.load("en_core_web_sm")
    nlp_tgt = spacy.load("es_core_news_sm")
    # nlp_src = spacy.load("en_core_web_trf")
    # nlp_tgt = spacy.load("es_dep_news_trf")

    doc_src = nlp_src(src_text)
    doc_tgt = nlp_tgt(tgt_text)

    # displacy.render(doc_src, style="dep", jupyter=True, options={"compact": True})
    # displacy.render(doc_tgt, style="dep", jupyter=True, options={"compact": True})
    # print spacy parse graph


    for token in doc_src:
        print(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_,
              [child for child in token.children])
    # for token in doc_tgt:
    #     print(token.text, token.dep_, token.head.text, token.head.pos_,
    #           [child for child in token.children])


' PUNCT punct 's AUX []
But CCONJ cc 's AUX []
it PRON nsubj 's AUX []
's AUX ROOT 's AUX [', But, it, use, now, ,, ', thought, !]
no DET det use NOUN []
use NOUN attr 's AUX [no]
now ADV advmod 's AUX []
, PUNCT punct 's AUX []
' PUNCT punct 's AUX []
thought VERB xcomp 's AUX [Alice, ,, ', pretend]
poor ADJ amod Alice PROPN []
Alice PROPN dobj thought VERB [poor]
, PUNCT punct thought VERB []
' PUNCT punct thought VERB []
to PART aux pretend VERB []
pretend VERB xcomp thought VERB [to, be]
to PART aux be AUX []
be AUX xcomp pretend VERB [to, people]
two NUM nummod people NOUN []
people NOUN attr be AUX [two]
! PUNCT punct 's AUX []


In [92]:
def find_leftmost_index(token):
    leftmost_index = token.i
    for child in token.children:
        if child.pos_ != 'NUM':  # Exclude numerals
            child_leftmost_index = find_leftmost_index(child)
            if child_leftmost_index < leftmost_index:
                leftmost_index = child_leftmost_index
    return leftmost_index

def modify_doc(doc):
    # Convert the Doc to a list of tokens
    token_list = list(doc)

    # Iterate over the tokens in the Doc
    for token in doc:
        # Check if the token is a noun
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN' or token.pos_ == 'PRON':
            # Get the leftmost index of the noun's children and subchildren
            leftmost_index = find_leftmost_index(token)

            # Move the noun and its children to the leftmost index
            noun_index = token.i
            children_indices = [child.i for child in token.subtree if child.i != noun_index and child.pos_ != 'NUM']

            # Remove the noun and its children from the token list
            token_list = [t for i, t in enumerate(token_list) if i not in [noun_index] + children_indices]

            # Insert the noun and its children at the leftmost index
            token_list.insert(leftmost_index, token)
            for child_index in children_indices:
                token_list.insert(leftmost_index + 1, doc[child_index])

    # Create a new Doc from the modified list of tokens
    print([token.text for token in token_list])
    new_doc = spacy.tokens.Doc(doc.vocab, words=[token.text for token in token_list])

    return new_doc

n_doc_src = nlp_src("But it's no use now, thought silly red Alice, to pretend to be two people!")
n_new_src_doc = modify_doc(n_doc_src)

print(n_new_src_doc)

# assert n_new_src_doc.text == "But it's no use now, thought Alice silly red , to pretend to be two people!"
# assert n_new_src_doc.text == "it But 's no use now , thought silly red Alice , to pretend to be two people ! "


# new_src_doc = modify_doc(doc_src)
# print(new_src_doc)

['But', 'it', "'s", 'use', 'no', 'now', ',', 'thought', 'Alice', 'red', 'silly', ',', 'to', 'pretend', 'to', 'be', 'two', 'people', '!']
But it 's use no now , thought Alice red silly , to pretend to be two people ! 


In [120]:
import spacy
from nltk import Tree

def tok_format(tok):
    return "_".join([tok.orth_, tok.pos_, tok.tag_])

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(tok_format(node), [to_nltk_tree(child) for child in node.children])
    else:
        return tok_format(node)

[to_nltk_tree(sent.root).pretty_print() for sent in n_doc_src.sents]
[to_nltk_tree(sent.root).pretty_print() for sent in doc_src.sents]
[to_nltk_tree(sent.root).pretty_print() for sent in new_src_doc.sents]
[to_nltk_tree(sent.root).pretty_print() for sent in doc_tgt.sents]

                                                       thought_VERB_VBD                                                                                 
     _________________________________________________________|_______________________________________________________                                   
    |         |         |                       |                                          |                   pretend_VERB_VB                          
    |         |         |                       |                                          |             _____________|____________                      
    |         |         |                   's_AUX_VBZ                              Alice_PROPN_NNP     |                      be_AUX_VB                
    |         |         |           ____________|____________________________              |            |              ____________|____________         
    |         |         |          |            |             |         use_NOU

ValueError: [E030] Sentence boundaries unset. You can add the 'sentencizer' component to the pipeline with: `nlp.add_pipe('sentencizer')`. Alternatively, add the dependency parser or sentence recognizer, or set sentence boundaries by setting `doc[i].is_sent_start`.

## Alternate method (doesn't know about sentence alignment)

In [7]:
%cd /workspace/novel-tongue/notebooks/alice_in_wonderland/

/workspace/novel-tongue/notebooks/alice_in_wonderland


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
with open(f"c1_en.txt", "r") as f:
    src_text = f.read()

with open(f"c1_es.txt", "r") as f:
    tgt_text = f.read()

nlp_src = spacy.load("en_core_web_sm")
nlp_tgt = spacy.load("es_core_news_sm")

doc_src = nlp_src(src_text)
doc_tgt = nlp_tgt(tgt_text)
sentences_src = list(doc_src.sents)
sentences_tgt = list(doc_tgt.sents)

displacy.render(sentences_src, style="dep", options={"compact": True})
displacy.render(sentences_tgt, style="dep", options={"compact": True})

In [163]:
def noun_phrase_syntax_en2es(phrase):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(phrase)
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

    spanish_syntax = []
    used_tokens = set()  # Using a set for faster look-up times

    for  token in doc:
        if token.pos_ == "NOUN" and token not in used_tokens:
            # Create a list to gather parts of the current noun phrase
            determiners = []
            ordinals = []
            adjectives = []

            # Iterate only over children of the noun for local determiners and adjectives
            for child in token.children:
                if child.pos_ == "DET" and child not in used_tokens:
                    determiners.append(child)
                elif child.pos_ == "ADJ" and parse_ordinal(child.text) and child not in used_tokens:
                    ordinals.append(child)
                elif child.pos_ == "ADJ" and child not in used_tokens:
                    adjectives.append(child)

            # Sort adjectives by their order in the original sentence
            adjectives_sorted = sorted(adjectives, key=lambda x: x.i)

            # Append elements to the syntax list in Spanish order
            spanish_syntax.extend(determiners)
            spanish_syntax.extend(ordinals)
            spanish_syntax.append(token)  # Append the noun
            spanish_syntax.extend(adjectives_sorted)  # Append adjectives in order of appearance

        elif token.pos_ not in ["NOUN", "ADJ", "DET"] and token not in used_tokens:
            spanish_syntax.append(token)
            used_tokens.add(token)

    # Recreate the sentence from the reordered tokens
    rebuilt_sentence = " ".join([token.text for token in spanish_syntax])
    return rebuilt_sentence


In [164]:
english_phrase = "We flew to the giant red center of the third blue earth"
spanish_syntax = noun_phrase_syntax_en2es(english_phrase)
print(spanish_syntax)
assert spanish_syntax == "We flew to the center giant red of the third earth blue"
# should actually be: "We flew to the giant center red of the third earth blue"

            flew_VERB_VBD                                                                 
      ____________|__________________________                                              
     |                                   to_ADP_IN                                        
     |                                       |                                             
     |                                 center_NOUN_NN                                     
     |             __________________________|_________________________                    
     |            |            |             |                     of_ADP_IN              
     |            |            |             |                         |                   
     |            |            |             |                   earth_NOUN_NN            
     |            |            |             |             ____________|____________       
We_PRON_PRP   the_DET_DT  giant_ADJ_JJ   red_ADJ_JJ   the_DET_DT  third_ADJ_JJ blue_A